In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

def selectkbest(indep_X, dep_Y, n):
    # Shift the data to ensure all values are non-negative
    indep_X_shifted = indep_X - indep_X.min().min()
    
    # Use chi2 after shifting the data
    test = SelectKBest(score_func=chi2, k=n)
    fit1 = test.fit(indep_X_shifted, dep_Y)
    selectk_features = fit1.transform(indep_X_shifted)
    selected_column = indep_X.columns[fit1.get_support(indices=True)]
    return selectk_features, selected_column


# Standard scalar function
def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)    
    return X_train, X_test, y_train, y_test

# Function to evaluate models and generate confusion matrix and accuracy report
def cm_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return accuracy, report, cm

# Logistic Regression
def logistic(X_train, y_train, X_test, y_test):       
    classifier = LogisticRegression(random_state=0)
    classifier.fit(X_train, y_train)
    accuracy, report, cm = cm_prediction(classifier, X_test, y_test)
    return accuracy, report, cm      

# SVM Linear Kernel
def svm_linear(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel='linear', random_state=0)
    classifier.fit(X_train, y_train)
    accuracy, report, cm = cm_prediction(classifier, X_test, y_test)
    return accuracy, report, cm

# SVM Non-linear Kernel
def svm_NL(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel='rbf', random_state=0)
    classifier.fit(X_train, y_train)
    accuracy, report, cm = cm_prediction(classifier, X_test, y_test)
    return accuracy, report, cm

# Naive Bayes
def navie(X_train, y_train, X_test, y_test):       
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    accuracy, report, cm = cm_prediction(classifier, X_test, y_test)
    return accuracy, report, cm

# K-Nearest Neighbors
def knn(X_train, y_train, X_test, y_test):
    classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
    classifier.fit(X_train, y_train)
    accuracy, report, cm = cm_prediction(classifier, X_test, y_test)
    return accuracy, report, cm

# Decision Tree Classifier
def decision(X_train, y_train, X_test, y_test):
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    accuracy, report, cm = cm_prediction(classifier, X_test, y_test)
    return accuracy, report, cm

# Random Forest Classifier
def random(X_train, y_train, X_test, y_test):
    classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    accuracy, report, cm = cm_prediction(classifier, X_test, y_test)
    return accuracy, report, cm

# Function to select features and apply models
def selectk_Classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    dataframe = pd.DataFrame(index=['ChiSquare'], columns=['Logistic', 'SVMl', 'SVMnl', 'KNN', 'Navie', 'Decision', 'Random'])
    min_length = min(len(acclog), len(accsvml), len(accsvmnl), len(accknn), len(accnav), len(accdes), len(accrf))
    
    for number in range(min_length):
        dataframe.loc['ChiSquare', 'Logistic'] = acclog[number]
        dataframe.loc['ChiSquare', 'SVMl'] = accsvml[number]
        dataframe.loc['ChiSquare', 'SVMnl'] = accsvmnl[number]
        dataframe.loc['ChiSquare', 'KNN'] = accknn[number]
        dataframe.loc['ChiSquare', 'Navie'] = accnav[number]
        dataframe.loc['ChiSquare', 'Decision'] = accdes[number]
        dataframe.loc['ChiSquare', 'Random'] = accrf[number]
    
    return dataframe

# Main Execution Flow
df1 = pd.read_csv("Preprocess_Bank_churners.csv", index_col=None)

# Convert Attrition_Flag to binary
y = np.where(df1['Attrition_Flag'] > 0, 1, 0)
X = df1.drop(columns=['Attrition_Flag'])

# Split data into train and test sets for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform feature selection (if needed)
kbest, selected_column = selectkbest(X, y, 7)

# Train and evaluate models
acclog = []
accsvml = []
accsvmnl = []
accknn = []
accnav = []
accdes = []
accrf = []

# Logistic Regression
accuracy, report, cm = logistic(X_train, y_train, X_test, y_test)
acclog.append(accuracy)

# SVM Linear
accuracy, report, cm = svm_linear(X_train, y_train, X_test, y_test)
accsvml.append(accuracy)

# SVM Non-linear
accuracy, report, cm = svm_NL(X_train, y_train, X_test, y_test)
accsvmnl.append(accuracy)

# K-Nearest Neighbors
accuracy, report, cm = knn(X_train, y_train, X_test, y_test)
accknn.append(accuracy)

# Naive Bayes
accuracy, report, cm = navie(X_train, y_train, X_test, y_test)
accnav.append(accuracy)

# Decision Tree
accuracy, report, cm = decision(X_train, y_train, X_test, y_test)
accdes.append(accuracy)

# Random Forest
accuracy, report, cm = random(X_train, y_train, X_test, y_test)
accrf.append(accuracy)

# Compile Results
result = selectk_Classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
print(result)

          Logistic      SVMl     SVMnl       KNN     Navie  Decision    Random
ChiSquare  0.90079  0.900296  0.924975  0.899309  0.874136  0.937808  0.958045
